In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,14812.21,-0.05,0.16,0.00,0.00,-0.01,0.00
3,-0.05,4039.39,-0.03,0.02,0.00,0.01,-0.00
4,0.16,-0.03,941.69,0.01,-0.00,0.00,-0.00
5,0.00,0.02,0.01,174.59,-0.00,-0.00,0.00
6,0.00,0.00,-0.00,-0.00,26.12,0.00,0.00
7,-0.01,0.01,0.00,-0.00,0.00,3.24,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.25


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00004,0.00000,0.00000,-0.00003,0.00000
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00005,-0.00002
4,0.00004,-0.00001,1.00000,0.00003,-0.00001,0.00002,-0.00000
5,0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00000,0.00001
6,0.00000,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00003
7,-0.00003,0.00005,0.00002,-0.00000,0.00002,1.00000,-0.00001
8,0.00000,-0.00002,-0.00000,0.00001,0.00003,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.23848946555333764

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.14163660e+07 2.26665006e+06 1.18219793e+05 3.81137278e+03
 7.59027916e+01 8.77253075e-01 5.00994795e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998748,-0.048890,-0.010448,-0.001931,-0.000289,-0.000036,-0.000003
1,0.048294,0.997557,-0.049705,-0.008627,-0.001288,-0.000159,-0.000013
2,0.012721,0.048662,0.997540,-0.048331,-0.006930,-0.000858,-0.000067
3,0.002937,0.010779,0.047403,0.997480,-0.051219,-0.006169,-0.000482
4,0.000584,0.002142,0.009195,0.050324,0.996685,-0.063035,-0.004816
5,0.000109,0.000397,0.001707,0.009217,0.062064,0.994939,-0.078460
6,0.000017,0.000063,0.000268,0.001447,0.009674,0.077998,0.996906


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00125236, 0.00244252, 0.00246023, 0.00251995, 0.00331481,
       0.00506075, 0.00309448])